# Использование хаба промптов для суммаризации книг
В качестве примера суммаризируем первые две главы книги Булгакова "Мастер и Маргарита".
## Инициализация GigaChat
> Обратите внимание, что нужно использовать большую модель гигачат, поддерживающую 8 тысяч токенов и более!

In [1]:
from langchain.prompts import load_prompt
from langchain.chains.summarize import load_summarize_chain
from langchain.chat_models import GigaChat

giga = GigaChat(credentials=..., model=...big_model...)

## Суммаризация книг, больших текстов
### Загрузка и подготовка данных

In [2]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

loader = TextLoader("../../../../docs/docs/use_cases/мастер_и_маргарита.txt")
documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 7000,
    chunk_overlap  = 0,
    length_function = len,
    is_separator_regex = False,
)
documents = text_splitter.split_documents(documents)
print(f"Количество частей книги: {len(documents)}")

Количество частей книги: 11


### Создание цепочки суммаризации

In [3]:
book_map_prompt = load_prompt("lc://prompts/summarize/map_reduce/summarize_book_map.yaml")
book_combine_prompt = load_prompt("lc://prompts/summarize/map_reduce/summarize_book_combine.yaml")

chain = load_summarize_chain(giga, chain_type="map_reduce", 
                             map_prompt=book_map_prompt,
                             combine_prompt=book_combine_prompt,
                             verbose=False)

In [6]:
res = chain.invoke({"input_documents": documents})

In [7]:
print(res["output_text"].replace(". ", ".\n"))

В этой части книги рассказывается о двух гражданах, появившихся на Патриарших прудах.
Михаил Берлиоз является председателем литературной ассоциации и редактором журнала, а Иван Бездомный — поэт.
После того, как они выпили абрикосовую воду, Берлиоз начал говорить о том, что Иисус Христос как личность не существовал, и все рассказы о нем — это миф.

В этой части книги описывается встреча Михаила Берлиоза и Ивана Бездомного с загадочным иностранцем, который внимательно слушал их разговор о неверии в бога и доказательствах его существования.
После того как иностранный гость узнал о том, что большинство людей в стране не верят в бога, он поблагодарил Берлиоза, что вызвало недоумение у Бездомного.

Профессор, представившийся специалистом по черной магии, прибыл в Москву для изучения рукописей чернокнижника Герберта Аврилакского.
Он вызвал интерес и недоверие у литераторов, но не стал раскрывать свои истинные намерения.

Пилат, прокуратор Иудеи, обсуждает с первосвященником Каифой вопрос о ка

Если нужно изменить количество предложений в суммаризации, то с помощью дополнительных параметров можно задать нужные объемы текста на этапах map и reduce.

In [12]:
res = chain.invoke({"input_documents": documents, "map_size": "одно предложение", "combine_size": "три предложения"})

In [11]:
print(res["output_text"].replace(". ", ".\n"))

В первой части книги рассказывается о встрече двух литераторов с иностранным путешественником, который предсказывает одному из них страшную судьбу.
Во второй части книги описывается беседа прокуратора Иудеи Понтия Пилата с арестованным Иешуа, которого обвиняют в оскорблении величества.
Пилат принимает решение не давать делу ход, но из-за опасений возможных волнений в Ершалаиме решает отправить Иешуа в заключение в Кесарию Стратоновой.
